In [1]:
import numpy as np
import pandas as pd
from scipy import optimize

In [2]:
folder = "../data/eval_data/"

path_list = []
for i in range(1, 23):
    filename = f"gaze_user{i}_eval_16.csv"
    path_list.append(folder+filename)

path_list

['../data/eval_data/gaze_user1_eval_16.csv',
 '../data/eval_data/gaze_user2_eval_16.csv',
 '../data/eval_data/gaze_user3_eval_16.csv',
 '../data/eval_data/gaze_user4_eval_16.csv',
 '../data/eval_data/gaze_user5_eval_16.csv',
 '../data/eval_data/gaze_user6_eval_16.csv',
 '../data/eval_data/gaze_user7_eval_16.csv',
 '../data/eval_data/gaze_user8_eval_16.csv',
 '../data/eval_data/gaze_user9_eval_16.csv',
 '../data/eval_data/gaze_user10_eval_16.csv',
 '../data/eval_data/gaze_user11_eval_16.csv',
 '../data/eval_data/gaze_user12_eval_16.csv',
 '../data/eval_data/gaze_user13_eval_16.csv',
 '../data/eval_data/gaze_user14_eval_16.csv',
 '../data/eval_data/gaze_user15_eval_16.csv',
 '../data/eval_data/gaze_user16_eval_16.csv',
 '../data/eval_data/gaze_user17_eval_16.csv',
 '../data/eval_data/gaze_user18_eval_16.csv',
 '../data/eval_data/gaze_user19_eval_16.csv',
 '../data/eval_data/gaze_user20_eval_16.csv',
 '../data/eval_data/gaze_user21_eval_16.csv',
 '../data/eval_data/gaze_user22_eval_16.csv

## Estimate visual axis by regression model

$p = a_1 + a_2 x + a_3 y$

$q = a_4 + a_5 x + a_6 y$

In [3]:
opt_param_list = np.empty([0, 6])
for i in range(22):
    print()
    print(f"path: {path_list[i]}")
    df = pd.read_csv(path_list[i])
    
    # gaze direction
    ray_x = df["ray_x"].values
    ray_y = df["ray_y"].values
    ray_z = df["ray_z"].values
    
    # gaze direction at z=1
    xe = ray_x[:] / ray_z[:] * 1.0
    ye = ray_y[:] / ray_z[:] * 1.0
    
    # marker position on a calibration plane at 1m away (HMD coordinates)
    p = df["u"].values
    q = df["v"].values
    
    # origin (HMD coordinates)
    eye_x = df["eye_x"].values
    eye_y = df["eye_y"].values
    eye_z = df["eye_z"].values
    
    # calibration function using regression model
    def calibrate_reg(param, x, y):
        ones = np.ones_like(x)
        zeros = np.zeros_like(x)
        x_lis = np.array([ones, x, y, zeros, zeros, zeros])
        y_lis = np.array([zeros, zeros, zeros, ones, x, y])
        x_calib = np.dot(param, x_lis)
        y_calib = np.dot(param, y_lis)
        return x_calib, y_calib
    
    # square error
    def err_square(param):
        # calibrate gaze direction
        calib_xe, calib_ye = calibrate_reg(param, xe, ye)
        
        # convert gaze position on calibration plane at 1m away (HMD coordinates)
        ones = np.ones_like(calib_xe)
        t = (ones - eye_z) / 1.0
        xh = eye_x + t * calib_xe
        yh = eye_y + t * calib_ye
    
        # calculate error
        err_x = xh - p
        err_y = yh - q
        return (err_x*err_x + err_y*err_y).sum()

    opt_param = optimize.fmin_cg(err_square, np.array([0.0, 1.0, 0.0, 0.0, 0.0, 1.0]))
    print(f"opt_param: {opt_param}")

    opt_param_list = np.vstack([opt_param_list, opt_param])

opt_param_list.shape


path: ../data/eval_data/gaze_user1_eval_16.csv
Optimization terminated successfully.
         Current function value: 0.085793
         Iterations: 4
         Function evaluations: 63
         Gradient evaluations: 9
opt_param: [ 0.00320711  1.13208233 -0.01222198  0.01455647  0.0386048   1.1010157 ]

path: ../data/eval_data/gaze_user2_eval_16.csv
Optimization terminated successfully.
         Current function value: 1.015571
         Iterations: 4
         Function evaluations: 63
         Gradient evaluations: 9
opt_param: [ 0.00803416  1.21816277  0.02074682 -0.12693028 -0.09146715  1.01628745]

path: ../data/eval_data/gaze_user3_eval_16.csv
Optimization terminated successfully.
         Current function value: 1.146869
         Iterations: 4
         Function evaluations: 63
         Gradient evaluations: 9
opt_param: [-0.00150704  1.18235419  0.04550453 -0.01459543 -0.06747103  0.9395894 ]

path: ../data/eval_data/gaze_user4_eval_16.csv
Optimization terminated successfully.
     

(22, 6)

In [4]:
df_param = pd.DataFrame(opt_param_list, columns=["p1", "p2", "p3", "p4", "p5", "p6"])
df_param.to_csv("../param/baseline_param.csv", index=False, encoding="utf_8_sig")